In [5]:
import pandas as pd
import random
import datetime

In [6]:
# function to create a list (of patients)

def createlist(num_pat):
    return list(range(1,num_pat + 1))

In [7]:
###############################
####### SET VARIABLES #########
###############################

# set number of patients to five
patient_list = createlist(5)

# choose patient for one, limited example 
patient_id = patient_list[0]

# entries to choose from

# regarding person
gender_list = [8532, 8507, 8521] # Female, Male, Other
birthyear_range = range(1920,2003)
race_list = [8515, 38003598, 38003614] # Asian, Black, European

#regarding visits, creating a list of all dates from 2020 and 2021
date1 = datetime.date(2020, 1, 1)
date2 = datetime.date(2021, 12, 31)
res_dates = [date1]
while date1 != date2:
    date1 += datetime.timedelta(days=1)
    res_dates.append(date1)
    
# regarding diagnoses and treatment
intensive_bin = [32037] # "Intensive Care"
beatm_bin = [4230167] # "Artificial respiration"
prone_bin = [4196006] # "Placing subject in prone position"
condition_list = [37311061, 444247, 4009307, 432870, 440417, 4195694] # "COVID-19"; "Venous Thrombosis"; "Heparin-induced thrombocytopenia with thrombosis"; "Thrombocytopenic disorder"; "Pulmonary Embolism"; "Acute respiratory distress syndrome" 
obs_list = [4169185, 4170358] # "Allergy to heparin"; "Allergy to heparinoid"
lab_list = [3048530, 3013466, 3029943] # "Fibrin D-dimer DDU [Mass/volume] in Platelet poor plasma"; "aPTT in Blood by Coagulation assay"; "Horowitz index in Arterial blood"; 
beatm_list = [3020716, 3017878, 21490650, 21490855] # "Inhaled oxygen concentration"; "Tidal volume.spontaneous+mechanical/Body weight [Volume/mass] --on ventilator"; "Pressure max Respiratory system airway --during inspiration"; "PEEP Respiratory system --on ventilator"
drug_list = [1367571, 1322207, 1301065, 1301025, 19001014, 19016072, 1315865] # "heparin"; "argatroban"; "dalteparin"; "enoxaparin"; "nadroparin"; "certoparin"; "fondaparinux"; 


In [8]:
patient_list

[1, 2, 3, 4, 5]

In [55]:
# define functions to create data

# create person
def create_person(person_id):
    person_id = person_id
    gender_concept_id = random.choice(gender_list)
    year_of_birth = random.choice(birthyear_range)
    month_of_birth = random.choice(range(1,12))
    if month_of_birth in [1, 3, 5, 7, 8, 10, 12]:
        day_of_birth = random.choice(range(1,31))
    elif month_of_birth in [4, 6, 9, 11]:
        day_of_birth = random.choice(range(1,30))
    elif month_of_birth == 2:
        day_of_birth = random.choice(range(1,28))
    else:
        pass
    race_concept_id = random.choice(race_list)
    ethnicity_concept_id = 0 # always set to zero!
    print("year_of_birth:", year_of_birth)
    print("month_of_birth:", month_of_birth)            
    print("day_of_birth:", day_of_birth)
    print("race_concept_id:", race_concept_id)
    print("ethnicity_concept_id:", ethnicity_concept_id)

# creating only entirely intensive-care and non-intensive care visits at the moment
def create_visit(person_id):
    person_id = person_id
    global visit_concept_id, visit_start_date, visit_end_date, visit_duration, visit_type_concept_id
    visit_concept_id = random.choice([9201, 32037]) # "Inpatient visit" or "Intensive care"
    visit_start_date = random.choice(res_dates)
    visit_end_date = visit_start_date + datetime.timedelta(days = random.choice(range(3,60)))
    visit_duration = visit_end_date - visit_start_date
    visit_type_concept_id = 32817
    print("person_id:", person_id)
    print("intensive:", intensive)
    print("visit_concept_id:", visit_concept_id)
    print("visit_start_date:", visit_start_date)
    print("visit_end_date:", visit_end_date)
    print("visit_duration:", visit_duration)
    print("visit_type_concept_id:", visit_type_concept_id)
    

In [53]:
# create drug exposures from drug list. i = number of boluses in case of LWMH/Fondaparinux or i = number of doserate changes in case of continuous infusions
# unsolved issue: dose/strength in case of continuous infusion unclear!
def create_drug_exp2(person_id, i):
    begin_drug = visit_start_date + datetime.timedelta(days=random.choice(range(4)))
    end_drug = begin_drug + (random.random() * (visit_end_date - begin_drug))
#    end_drug = begin_drug + datetime.timedelta(days = random.choice(range(2,30)))
    # create dttm from date, add random number of hours to startdate midnight
    drug_exposure_start_datetime = datetime.datetime.combine(begin_drug, datetime.datetime.min.time()) + datetime.timedelta(hours = random.choice(range(12)))    
    drug_exposure_id = 0
    drug_concept_id = random.choice(drug_list)
    if drug_concept_id in [1367571, 1301065, 19001014, 19016072]:
        drug_unit = 8510
    elif drug_concept_id in [1322207, 1301025, 1315865]:
        drug_unit = 8576
    else:
        pass
    ingredient_concept_id = drug_concept_id
    valid_start_date = datetime.date(1970, 1, 1)
    # continuous infusion of drugs, quantity referring to [dose] / h 
    if drug_concept_id in [1367571, 1322207]:
        drug_exposure_end_datetime = drug_exposure_start_datetime
        for x in range(i):
            drug_exposure_id += 1 
            drug_exposure_start_datetime = drug_exposure_end_datetime + datetime.timedelta(hours=random.choice(range(2)))
            drug_exposure_start_date = drug_exposure_start_datetime.date() # create date from dttm
            if drug_concept_id == 1367571: # heparin in IE/h
                quantity = random.choice(range(200, 900, 100))
            elif drug_concept_id == 1322207: # argatroban in mg/h
                quantity = random.choice(range(5, 10, 1))
            else:
                pass
            drug_exposure_end_datetime = drug_exposure_start_datetime + datetime.timedelta(hours=random.choice(range(12)))
            drug_exposure_end_date = drug_exposure_end_datetime.date()
            if drug_exposure_start_date > end_drug:
                break
            print("drug_concept_id:", drug_concept_id)
            print("ingredient_concept_id:", ingredient_concept_id)
            print("drug_exposure_id:", drug_exposure_id)
            print("drug_exposure_start_date:", drug_exposure_start_date)
            print("drug_exposure_start_datetime:", drug_exposure_start_datetime)
            print("quantity:", quantity)
            print("drug_exposure_end_datetime:", drug_exposure_end_datetime)
            print("drug_exposure_end_date:", drug_exposure_end_date)
            print("end_drug:", end_drug)
    if drug_concept_id in [1301065, 19001014, 19016072, 1301025, 1315865]:
        for x in range(i):
            drug_exposure_id += 1 
            drug_exposure_start_datetime += datetime.timedelta(hours=random.choice(range(4,20)))
            drug_exposure_start_date = drug_exposure_start_datetime.date() # create date from dttm
            if drug_concept_id in [1301065, 19001014]: # dalteparin, nadroparin
                quantity = random.choice(range(3000, 15000, 1000))
            elif drug_concept_id == 19016072: # certoparin
                quantity = random.choice(range(1000, 4000, 500))
            elif drug_concept_id == 1301025: # enoxaparin
                quantity = random.choice(range(10,120,10))
            elif drug_concept_id == 1315865: # fondaparinux
                quantity = random.choice(range(1,4,1))
            else:
                pass
            drug_exposure_end_date = drug_exposure_start_date
            if drug_exposure_start_date > end_drug:
                break
            print("drug_concept_id:", drug_concept_id)
            print("ingredient_concept_id:", ingredient_concept_id)
            print("drug_exposure_id:", drug_exposure_id)
            print("drug_exposure_start_date:", drug_exposure_start_date)
            print("drug_exposure_start_datetime:", drug_exposure_start_datetime)
            print("quantity:", quantity)
            print("drug_exposure_end_date:", drug_exposure_end_date)
            print("end_drug:", end_drug)
    else:
        pass

In [59]:
# call functions
create_person(1)
create_visit(1)
create_drug_exp2(1, 10)



year_of_birth: 1978
month_of_birth: 2
day_of_birth: 7
race_concept_id: 38003598
ethnicity_concept_id: 0
person_id: 1
intensive: 1
visit_concept_id: 32037
visit_start_date: 2021-12-17
visit_end_date: 2021-12-30
visit_duration: 13 days, 0:00:00
visit_type_concept_id: 32817
drug_concept_id: 1315865
ingredient_concept_id: 1315865
drug_exposure_id: 1
drug_exposure_start_date: 2021-12-21
drug_exposure_start_datetime: 2021-12-21 02:00:00
quantity: 3
drug_exposure_end_date: 2021-12-21
end_drug: 2021-12-24
drug_concept_id: 1315865
ingredient_concept_id: 1315865
drug_exposure_id: 2
drug_exposure_start_date: 2021-12-21
drug_exposure_start_datetime: 2021-12-21 20:00:00
quantity: 1
drug_exposure_end_date: 2021-12-21
end_drug: 2021-12-24
drug_concept_id: 1315865
ingredient_concept_id: 1315865
drug_exposure_id: 3
drug_exposure_start_date: 2021-12-22
drug_exposure_start_datetime: 2021-12-22 08:00:00
quantity: 2
drug_exposure_end_date: 2021-12-22
end_drug: 2021-12-24
drug_concept_id: 1315865
ingredient